<a href="https://colab.research.google.com/github/PrakharU08/QK-internship/blob/main/QK_Question_and_Answer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers

So transforms role in BERT model is to identify the relation bertween diffrent words and in the BERT model B stands for bidirectional Which means it tries ti understand both side of a sentence and create a perspective 
For ex: Raj withdraws Rs100 from bank  ;  Raj lives at the bank of a river

In [2]:
import torch

For Question Answering we use the `BertForQuestionAnswering` class from the `transformers` library.

This class supports fine-tuning, but for this example we will keep things simpler and load a BERT model that has already been fine-tuned for the SQuAD benchmark.

The `transformers` library has a large collection of pre-trained models which you can reference by name and load easily. The full list is in their documentation [here](https://huggingface.co/transformers/pretrained_models.html).

For Question Answering, they have a version of BERT-large that has already been fine-tuned for the SQuAD benchmark. 

BERT-large is really big... it has 24-layers and an embedding size of 1,024, for a total of 340M parameters! Altogether it is 1.34GB, so expect it to take a couple minutes to download to your Colab instance. 

(Note that this download is not using your own network bandwidth--it's between the Google instance and wherever the model is stored on the web).

Note: I believe this model was trained on version 1 of SQuAD, since it's not outputting whether the question is "impossible" to answer from the text (which is part of the task in v2 of SQuAD).


In [3]:
from transformers import BertForQuestionAnswering

model = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')


Load the tokenizer as well. 

Side note: Apparently the vocabulary of this model is identicaly to the one in bert-base-uncased. You can load the tokenizer from `bert-base-uncased` and that works just as well.

In [4]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

# Question And Answer

Now we're ready to feed in an example!

A QA example consists of a question and a passage of text containing the answer to that question.

Let's try an example using the text in this tutorial!

In [7]:
question = "what is Thought Leadership?"
answer_text = """Our company is dedicated to maintaining a core set of values in all of our business dealings. They are:Integrity: We are true to our word. We treat our clients with respect. We offer solutions that maintain system security.
Result Oriented: We strive to provide solutions that make business easier and deliver the results needed to succeed even in the face of unpredictability.
Thought Leadership: Our assurance experts have their finger on the pulse of Quality Assurance so our services evolve to meet the needs of our valued Clients.
Effective Relationship: Teamwork is the key to solving problems, addressing issues, developing new processes, and assuring that our valued Clients receive the level of support they deserve.
Initiative: As problem solvers, it is our responsibility to take the initiative to provide our clients with the services they need to meet new and ongoing compliance requirements."""

In [8]:
# Apply the tokenizer to the input text, treating them as a text-pair.
input_ids = tokenizer.encode(question, answer_text)

print('The input has a total of {:} tokens.'.format(len(input_ids)))

The input has a total of 176 tokens.


Just to see exactly what the tokenizer is doing, let's print out the tokens with their IDs.

In [9]:
# BERT only needs the token IDs, but for the purpose of inspecting the 
# tokenizer's behavior, let's also get the token strings and display them.
tokens = tokenizer.convert_ids_to_tokens(input_ids)

# For each token and its id...
for token, id in zip(tokens, input_ids):
    
    # If this is the [SEP] token, add some space around it to make it stand out.
    if id == tokenizer.sep_token_id:
        print('')
    
    # Print the token string and its ID in two columns.
    print('{:<12} {:>6,}'.format(token, id))

    if id == tokenizer.sep_token_id:
        print('')
    

[CLS]           101
what          2,054
is            2,003
thought       2,245
leadership    4,105
?             1,029

[SEP]           102

our           2,256
company       2,194
is            2,003
dedicated     4,056
to            2,000
maintaining   8,498
a             1,037
core          4,563
set           2,275
of            1,997
values        5,300
in            1,999
all           2,035
of            1,997
our           2,256
business      2,449
dealings     24,069
.             1,012
they          2,027
are           2,024
:             1,024
integrity    11,109
:             1,024
we            2,057
are           2,024
true          2,995
to            2,000
our           2,256
word          2,773
.             1,012
we            2,057
treat         7,438
our           2,256
clients       7,846
with          2,007
respect       4,847
.             1,012
we            2,057
offer         3,749
solutions     7,300
that          2,008
maintain      5,441
system        2,29

We've concatenated the `question` and `answer_text` together, but BERT still needs a way to distinguish them. BERT has two special "Segment" embeddings, one for segment "A" and one for segment "B". Before the word embeddings go into the BERT layers, the segment A embedding needs to be added to the `question` tokens, and the segment B embedding needs to be added to each of the `answer_text` tokens. 

These additions are handled for us by the `transformer` library, and all we need to do is specify a '0' or '1' for each token. 

Note: In the `transformers` library, huggingface likes to call these `token_type_ids`, but I'm going with `segment_ids` since this seems clearer, and is consistent with the BERT paper.

In [10]:
# Search the input_ids for the first instance of the `[SEP]` token.
sep_index = input_ids.index(tokenizer.sep_token_id)

# The number of segment A tokens includes the [SEP] token istelf.
num_seg_a = sep_index + 1

# The remainder are segment B.
num_seg_b = len(input_ids) - num_seg_a

# Construct the list of 0s and 1s.
segment_ids = [0]*num_seg_a + [1]*num_seg_b

# There should be a segment_id for every input token.
assert len(segment_ids) == len(input_ids)

We're ready to feed our example into the model!



In [11]:
# Run our example through the model.
start_scores, end_scores = model(torch.tensor([input_ids]), # The tokens representing our input text.
                                 token_type_ids=torch.tensor([segment_ids])) # The segment IDs to differentiate question from answer_text


In [14]:
# Find the tokens with the highest `start` and `end` scores.
answer_start = torch.argmax(start_scores)
answer_end = torch.argmax(end_scores)

# Combine the tokens in the answer and print it out.
answer = ' '.join(tokens[answer_start:answer_end+1])


print("Question : " +question)
print('Answer : "' + answer + '"')

Question : what is Thought Leadership?
Answer : "our assurance experts have their finger on the pulse of quality assurance"
